In [50]:
import spacy
import pandas as pd
from bs4 import BeautifulSoup
import re
import os
PATH = "/home/samy/csv_pickle_parquet/html"

data_html_page = pd.DataFrame(columns = [
    "character","html_page"
])

data_characters = pd.DataFrame(
    columns=[
        "name_title",
        "name_infobox",
        "aliases",
        "text_length",
        "books",
        "nature_of_appearance",
    ]
)



data_character = pd.DataFrame(columns = [])

In [51]:
for dirpath,dirnames,filenames in os.walk(PATH):
    for filename in filenames:
        if filename.endswith(".html"):
            filepath = os.path.join(dirpath,filename)
            with open(filepath,"r") as fp:
                html = fp.read()
                data_html_page.loc[len(data_html_page),:] = [filename,html]
                

In [52]:
data_html_page

,character,html_page
0,Kedry.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
1,Bessa.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
2,Addam.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
3,Daeryssa.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
4,Harlen_Tyrell.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
...,...,...
2043,Matthos_Seaworth.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
2044,Murmison.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
2045,Owen_(brother_of_Meribald).html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."
2046,Porther.html,"<!DOCTYPE html>\n<html class=""client-nojs"" lan..."


In [70]:
BOOKS_TITLE = {
    "A Game of Thrones": "GOT1.txt",
    "A Clash of Kings": "GOT2.txt",
    "A Storm of Swords": "GOT3.txt",
    "A Feast for Crows": "GOT4.txt",
    "A Dance with Dragons": "GOT5.txt",
}



def get_aliases(soup):
    aliases_list = []
    aliases_html_th = soup.find("table", class_="infobox").find("th", text="Aliases")
    if aliases_html_th:
        aliases_html_td = aliases_html_th.find_next("td").find_all("li")
        for elmt in aliases_html_td:
            name = elmt.text
            name = (
                "".join([i for i in name if not i.isdigit()])
                .replace("[", "")
                .replace("]", "")
            )
            aliases_list.append(name)
    return aliases_list


def get_alias(soup):
    name = []
    alias_html_th = soup.find("table", class_="infobox").find("th", text="Alias")
    if alias_html_th:
        alias = alias_html_th.find_next("td").text
        alias = (
            "".join([i for i in alias if not i.isdigit()])
            .replace("[", "")
            .replace("]", "")
            .strip()
        )
        name.append(alias)
    return name


def get_title_name(soup): #Return the title of the html page of a souped html file
    title_html_h1 = soup.find("h1").text
    return title_html_h1


def get_infobox_name(soup): #Return the infobox name of the html page of a souped html file
    infobox = soup.find("table", class_="infobox") 
    return infobox
    sub_table = infobox.find("table")
    if sub_table:
        infobox_name = sub_table.find_all("td")[1]
        span = infobox_name.find("span")
        if span:
            span.decompose()
        return infobox_name.text
    else:
        return infobox.find_next("th").text


def get_text_length(soup): #Get the length of text of the html file
    text_list = soup.find_all("p")
    text = "".join(i.text for i in text_list).replace("\n", "")
    text = re.sub(r"\[\d+\]", "", text)
    return len(text)


def get_books(soup):
    book_list = []
    books_html_th = soup.find("table", class_="infobox").find("th", text="Books")
    if books_html_th:
        books_html_td = books_html_th.find_next("td").find_all("li")
        for elmt in books_html_td:
            name = elmt.text
            book_list.append(name)
    return book_list


def get_book(soup):
    name = []
    book_html_th = soup.find("table", class_="infobox").find("th", text="Book")
    if book_html_th:
        book = book_html_th.find_next("td").text
        book = (
            "".join([i for i in book if not i.isdigit()])
            .replace("[", "")
            .replace("]", "")
            .strip()
        )
        name.append(book)
    return name


def association_name_number(book_list, app_list):
    new_book_list = list()
    for i in range(len(book_list)):
        book_clean = book_list[i].strip()
        if book_clean in BOOKS_TITLE:
            new_book_list.append(BOOKS_TITLE[book_clean])
        else:
            app_list[i] = ""
    return new_book_list


def split_name_appearance(book_app_list):
    book_name_list = [element.split("(")[0] for element in book_app_list]
    app_list = [element.split("(")[1].replace(")", "") for element in book_app_list]
    book_list = association_name_number(book_name_list, app_list)
    new_app_list = []
    for elmt in app_list:
        if elmt:
            new_app_list.append(elmt)
    return book_list, new_app_list




def get_total_information(page_name):
    html = data_html_page[data_html_page["character"]==page_name].html_page.unique()[0]
    soup = BeautifulSoup(html)

    book_list = get_books(soup) + get_book(soup)
    books, appearances = split_name_appearance(book_list)

    return {
        "name_title": get_title_name(soup),
        "name_infobox": get_infobox_name(soup),
        "aliases": get_aliases(soup) + get_alias(soup),
        "page_rank": None,
        "text_length": get_text_length(soup),
        "books": books,
        "nature_of_appearance": appearances,
    }

In [79]:
get_total_information("Eddard_Stark.html")["name_infobox"].find("tbody")

<tbody><tr><td class="infobox-image" colspan="2" style="text-align:center">
<a class="image" href="/index.php/File:Eddard_Amoka.jpg"><img alt="Eddard Amoka.jpg" height="462" src="/images/0/0a/Eddard_Amoka.jpg" width="300"/></a><div>Eddard Stark by <a class="external text" href="http://en.amokanet.ru/gallery/martin/martin.html" rel="nofollow">Amok©</a></div></td></tr><tr><td colspan="2" style="text-align:center">
<hr/></td></tr><tr><th scope="row">Alias</th><td>
Ned<sup class="reference" id="cite_ref-Ragot2.7B.7B.7B3.7D.7D.7D_1-0"><a href="#cite_note-Ragot2.7B.7B.7B3.7D.7D.7D-1">[1]</a></sup><br/>The Ned<sup class="reference" id="cite_ref-Radwd62.7B.7B.7B3.7D.7D.7D_2-0"><a href="#cite_note-Radwd62.7B.7B.7B3.7D.7D.7D-2">[2]</a></sup><br/>The quiet wolf<sup class="reference" id="cite_ref-Rasos24.7B.7B.7B3.7D.7D.7D_3-0"><a href="#cite_note-Rasos24.7B.7B.7B3.7D.7D.7D-3">[3]</a></sup></td></tr><tr><th scope="row">Title</th><td>
<a href="/index.php/Lord_of_Winterfell" title="Lord of Winterfel